In [1]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

OPENAI_API_KEY is set.
🚀 Setup complete. Continue to the next cell.


In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
import json


def search_internet(query: str) -> str:
    """
    Searches the internet using Google Serper API.

    Args:
        query: The search query.

    Returns:
        str: The search results in JSON format.
    """
    search = GoogleSerperAPIWrapper()
    search_results = search.results(query)
    search_results_json = json.dumps(search_results, indent=4)
    return search_results_json

In [ ]:
from openai import OpenAI
import json
from copy import deepcopy

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_internet",
            "description": "Searches the internet with the given query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query."},
                },
                "required": ["query"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

TOOL_MAP = {
    "search_internet": search_internet,
}


def process_tools_and_update_messages(
    messages, tool_calls, last_message, show_tool_outputs=False
):
    """
    Processes the tools and updates the messages.

    Args:
        messages: The messages to update.
        tool_calls: The tool calls to process.

    Returns:
        list: The updated messages.
    """
    print("BEFORE: ", messages)
    updated_messages = deepcopy(messages)

    for tool_call in tool_calls:
        name = tool_call.function.name
        if show_tool_outputs:
            print(f"Processing tool call: {name}")
        args = json.loads(tool_call.function.arguments)
        func = TOOL_MAP.get(name)

        if not func:
            print(f"Function {name} not found.")
            continue

        # call the tool function
        tool_output = func(**args)

        if show_tool_outputs:
            print(f"Tool output: {tool_output}")

        # Append the assistant's function call message
        updated_messages.append(last_message)
        updated_messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_output),
            }
        )
    print("AFTER: ", updated_messages)
    return updated_messages


def llm(prompt: str, max_tool_iters=10, show_tool_outputs=False) -> str:
    """
    Calls the OpenAI language model with the given prompt.

    Args:
        prompt: The prompt to pass to the language model.
        max_tokens: The maximum number of tokens to generate.

    Returns:
        str: The response from the language model.
    """

    messages = [
        {
            "role": "system",
            "content": f"""
                        You are a helpful assistant. 
                        Use the provided tools to assist the user if required.
                        TOOLS: {json.dumps(tools, indent=4)}
                        """,
        },
        {"role": "user", "content": prompt},
    ]

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
    )
    last_message = response.choices[0].message
    tool_calls = response.choices[0].message.tool_calls
    messages = process_tools_and_update_messages(
        messages,
        tool_calls,
        last_message=last_message,
        show_tool_outputs=show_tool_outputs,
    )
    # iters = 0

    # while tool_calls and len(tool_calls) > 0 and iters < max_tool_iters:
    #     if show_tool_outputs:
    #         print(f"Processing tool calls. Iteration: {iters}.")
    #         print(f"Tool calls: {tool_calls}")

    #     response = openai.chat.completions.create(
    #         model="gpt-4o",
    #         messages=messages,
    #         tools=tools,
    #     )
    #     tool_calls = response.choices[0].message.tool_calls
    #     last_message = response.choices[0].message
    #     messages = process_tools_and_update_messages(
    #         messages,
    #         tool_calls,
    #         last_message=last_message,
    #         show_tool_outputs=show_tool_outputs,
    #     )
    #     iters += 1

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
    )

    return response.choices[0].message.content

In [46]:
query = "What is the time right now in NYC and Mumbai?"

response = llm(query, show_tool_outputs=True)

print(response)

BEFORE:  [{'role': 'system', 'content': '\n                        You are a helpful assistant. \n                        Use the provided tools to assist the user if required.\n                        TOOLS: [\n    {\n        "type": "function",\n        "function": {\n            "name": "search_internet",\n            "description": "Searches the internet with the given query.",\n            "parameters": {\n                "type": "object",\n                "properties": {\n                    "query": {\n                        "type": "string",\n                        "description": "The search query."\n                    }\n                },\n                "required": [\n                    "query"\n                ],\n                "additionalProperties": false\n            },\n            "strict": true\n        }\n    }\n]\n                        '}, {'role': 'user', 'content': 'What is the time right now in NYC and Mumbai?'}]
Processing tool call: search_internet
Too